In [40]:
import torch

In [47]:
rewards = torch.rand(4, 5, dtype=torch.float16)

In [45]:
rewards - rewards.mean(1, keepdim=True) / rewards.std(1, keepdim=True)

tensor([[-0.4924, -1.1812, -0.9333, -1.3558, -0.4794],
        [-0.7612, -0.7557, -0.9516, -0.2712, -0.9327],
        [-1.8456, -1.2907, -1.8423, -1.2641, -1.4576],
        [-0.8187, -0.6733, -0.8718, -1.2719, -1.1838]])

In [1]:
import torchtune
import os, sys
sys.path.insert(0, "..")
from recipes.r1_full_single_device import FullRLRecipeSingleDevice
from omegaconf import OmegaConf

In [2]:
cfg = OmegaConf.load("../recipes/configs/llama3_2/3B_full_rl_single_device_cuda.yaml")

In [3]:
recipe = FullRLRecipeSingleDevice(cfg)

INFO:torchtune.utils._logging:Hint: enable_activation_checkpointing is True, but enable_activation_offloading isn't. Enabling activation offloading should reduce memory further.
DEBUG:torchtune.utils._logging:Setting manual seed to local seed 3763484870. Local seed is seed + rank = 3763484870 + 0


In [4]:
recipe.setup(cfg=cfg)

Writing logs to /checkpoint/kwiat/tmp/full-llama3.2-finetune/log_1738012677.txt


INFO:torchtune.utils._logging:Model is initialized with precision torch.bfloat16.
INFO:torchtune.utils._logging:Memory stats after model init:
	GPU peak memory allocation: 12.09 GiB
	GPU peak memory reserved: 12.19 GiB
	GPU peak memory active: 12.09 GiB
INFO:torchtune.utils._logging:Tokenizer is initialized from file.
INFO:torchtune.utils._logging:In-backward optimizers are set up.
INFO:torchtune.utils._logging:Loss is initialized.
INFO:torchtune.utils._logging:Dataset and Sampler are initialized.
INFO:torchtune.utils._logging:No learning rate scheduler configured. Using constant learning rate.
INFO:torchtune.utils._logging: Profiler config after instantiation: {'enabled': False}


In [5]:
for i, batch in enumerate(recipe._dataloader):
    print(i)
    print(batch)
    break

0
{'tokens': tensor([[128000,     32,  10652,   1990,   2724,    323,  22103,     13,    578,
           1217,  17501,    264,   3488,     11,    323,    279,  22103,  68577,
            433,     13,    578,  18328,   1176,  15849,    922,    279,  33811,
           1920,    304,    279,   4059,    323,   1243,   5825,    279,   1217,
            449,    279,   4320,     13,    578,  33811,   1920,    323,   4320,
            527,  44910,   2949,    366,  27963,   1500,  27963,     29,    323,
            366,   9399,   1500,   9399,     29,   9681,     11,  15947,     11,
            602,   1770,   2637,    366,  27963,     29,  20489,    287,   1920,
           1618,    524,  27963,     29,    366,   9399,     29,   9399,   1618,
            524,   9399,  14611,   2724,     25,  55357,   6944,    311,   3780,
            264,  13339,   4447,    430,   7194,    220,     17,  28518,   2734,
          14801,     13,  55357,    706,    220,   4370,   3117,  58174,     11,
            323

In [6]:
tokens = batch["tokens"].to("cuda")

In [14]:
batch_tokens = tokens[:, None, :].expand(-1, 3, -1)

In [15]:
batch_tokens.shape

torch.Size([4, 3, 192])

In [20]:
batch_tokens.reshape(4*3, 192)

tensor([[128000,     32,  10652,  ..., 128004, 128004, 128004],
        [128000,     32,  10652,  ..., 128004, 128004, 128004],
        [128000,     32,  10652,  ..., 128004, 128004, 128004],
        ...,
        [128000,     32,  10652,  ..., 128004, 128004, 128004],
        [128000,     32,  10652,  ..., 128004, 128004, 128004],
        [128000,     32,  10652,  ..., 128004, 128004, 128004]],
       device='cuda:0')

In [21]:
query_responses, logits = recipe.generate_trajectory(batch_tokens.reshape(4*3, -1))

In [25]:
query_responses = query_responses.reshape(4, 3, -1)
logits = logits.reshape(4, 3, -1)

In [28]:
c1, c2, c3 = query_responses[0].tolist()

In [35]:
recipe._tokenizer.encode("Hello, world!")

[128000, 9906, 11, 1917, 0, 128001]

In [37]:
recipe._tokenizer.decode([0], False, False)

'!'

In [39]:
recipe._tokenizer.decode(recipe._tokenizer.stop_tokens, False, False)

'<|end_of_text|><|eot_id|><|eom_id|>'

In [34]:
recipe._tokenizer.decode(c2, False, False)

"<|begin_of_text|>A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <think></think> and <answer></answer> tags, respectively, i.e., <think>reasoning process here</think> <answer>answer here</answer>. User: Gerald wants to buy a meat pie that costs 2 pfennigs. Gerald has 54 farthings, and there are 6 farthings to a pfennig. How many pfennigs will Gerald have left after buying the pie?. Assistant:<|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_righ

In [13]:
batch_size, context_length = tokens.shape

In [14]:
responses = query_responses[:, context_length:].clone()

In [30]:
recipe._tokenizer.decode(batch["tokens"][1].tolist())

'A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <think></think> and <answer></answer> tags, respectively, i.e., <think>reasoning process here</think> <answer>answer here</answer>. User: A fifth-grade class went on a field trip to the zoo, and their class of 10 students merged with another class with the same amount of students. 5 parents offered to be a chaperone, and 2 of the teachers from both classes will be there too. When the school day was over, the students could choose to go home and 10 of them left. Two of the chaperones were parents in that group, so they left as well. How many individuals were left at the zoo?. Assistant:'

In [31]:
recipe._tokenizer.decode(responses[1].tolist())

' <think>First, I have to find out how many people left the zoo. That was 10 + 2 - 10 - 2 = 0.</think> <answer>There were 0 individuals left at the zoo.</answer><|end_of_text|><|begin_of_text|>The number of individuals left at the zoo is 0. 5<|end_of_text|><|begin_of_text|>This is because 10 parents offered to be a chaperone, and 2 of the teachers from both classes will be there too. Two of the'

In [38]:
query_response_padding_masks = (query_responses != recipe._tokenizer.pad_id)

In [39]:
from torchtune import generation

masks = generation.get_causal_mask_from_padding_mask(query_response_padding_masks)

In [41]:
position_ids = generation.get_position_ids_from_padding_mask(
            query_response_padding_masks
        )

In [42]:
logits = logits[:, context_length-1:]

In [44]:
from torchtune import rlhf

logprobs = rlhf.logits_to_logprobs(logits, responses, 1.0)

In [46]:
logprobs.shape

torch.Size([4, 100])

In [48]:
ref_logits = recipe._model(query_responses.clone(), input_pos=position_ids, mask=masks)

In [50]:
ref_logits = rlhf.truncate_sequence_for_logprobs(ref_logits, context_length)
ref_logprobs = rlhf.logits_to_logprobs(ref_logits, responses, 1.0)

In [9]:
batch["answers"]

['7', '15', '16', '32']

In [10]:
recipe._model.tok_embeddings

Embedding(128256, 3072)

In [35]:
recipe._tokenizer.decode(out_tokens[0].tolist(), False, False)

'<|begin_of_text|>A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <think></think> and <answer></answer> tags, respectively, i.e., <think>reasoning process here</think> <answer>answer here</answer>. User: Gerald wants to buy a meat pie that costs 2 pfennigs. Gerald has 54 farthings, and there are 6 farthings to a pfennig. How many pfennigs will Gerald have left after buying the pie?. Assistant:<|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_righ

In [45]:
out_tokens[0][150]

tensor(128004, device='mps:0')

In [49]:
out_logits[0, 152].argmax()

tensor(13, device='mps:0')

In [20]:
batch

{'tokens': tensor([[128000,     32,  10652,   1990,   2724,    323,  22103,     13,    578,
            1217,  17501,    264,   3488,     11,    323,    279,  22103,  68577,
             433,     13,    578,  18328,   1176,  15849,    922,    279,  33811,
            1920,    304,    279,   4059,    323,   1243,   5825,    279,   1217,
             449,    279,   4320,     13,    578,  33811,   1920,    323,   4320,
             527,  44910,   2949,    366,  27963,   1500,  27963,     29,    323,
             366,   9399,   1500,   9399,     29,   9681,     11,  15947,     11,
             602,   1770,   2637,    366,  27963,     29,  20489,    287,   1920,
            1618,    524,  27963,     29,    366,   9399,     29,   9399,   1618,
             524,   9399,  14611,   2724,     25,  55357,   6944,    311,   3780,
             264,  13339,   4447,    430,   7194,    220,     17,  28518,   2734,
           14801,     13,  55357,    706,    220,   4370,   3117,  58174,     11,
      

# Next TODO:
- [ ] Properly format the prompt in the data
- [ ] Do the RL training loop

RESEARCH IDEA:
Continuous RL for CoT? instead of predicting tokens, we predict the continuous embeddings